In [27]:
import pandas as pd
from openai import OpenAI
import json
import time

# 配置 OpenAI 客户端
client = OpenAI(base_url="https://chatapi.littlewheat.com/v1", api_key="sk-w33LIKFEXz7UHKg2R5tVvboSNAU76iCZsvyOyOjPtLLPp89S")  

# 读取 Excel 文件
file_path = r"C:\Users\86155\Downloads\The New York Times - Search (1).xlsx"  
df = pd.read_excel(file_path)

empty_rows = df[df.isnull().all(axis=1)].index
first_empty_row = empty_rows[0] if not empty_rows.empty else len(df)
#df = df.iloc[:10]

# 添加问题一到问题七的列
questions = [
    "问题一", "问题二", "问题三", "问题四", "问题五", "问题六", "问题七"
]
for question in questions:
    df[question] = ""

# 提问内容
questions_prompt = """
请分别回答以下问题：
- 问题一：提取新闻发生的最主要地点，最多不超过10字，如果提取不出来就输出为加勒比地区，结尾不加表点符号。
- 问题二：概况新闻主要描述内容，10个字以内，结尾不加表点符号，概括不出来就输出未知。
- 问题三：概况新闻事件起因（从文章开头获取），10个字以内，结尾不加表点符号，概括不出来就输出未知。
- 问题四：概况新闻事件结果（从文章结尾获取），10个字以内，结尾不加表点符号，概括不出来就输出未知。
- 问题五：概况新闻事件影响（从文章结尾获取），10个字以内，结尾不加表点符号，概括不出来就输出未知。
- 问题六：请根据以下分类体系对标题进行分类（概括不出来就输出未知）：
        - 自然灾害、气候、自然资源、地理
        - 军事装备、军事工业、军事规模、政策法规
        - 贸易、健康、能源、工业、农业、服务业
        - 人口、医疗、教育、交通、就业失业、收入、社会保障
        - 历史人文、公共文化、河流文化
        - 政治
        - 外交
        - 宗教
        - 科技
- 问题七：将内容扩充至200字以上，要求内容与加勒比地区有关，并且只有一个段落不分行。
请严格返回 JSON 格式的结果，返回格式：{"问题一": "xxx", "问题二": "xxx", "问题三": "xxx", "问题四": "xxx", "问题五": "xxx", "问题六": "xxx", "问题七": "xxx"}
"""

# 调用 GPT API 获取问题回答
def call_gpt_api(message, max_token, retries):
    for _ in range(retries):
        try:
            response = client.chat.completions.create(
                model="gpt-3.5-turbo",
                messages=message,
                max_tokens=max_token,
                timeout=30
            )
            return response.choices[0].message.content.strip()
        except Exception as e:
            print(f"API调用失败: {e}, 重试中...")
            time.sleep(1)  # 未响应重试
    return ""

# 获取问题答案
def get_answers(text):
    kinds = [
        {"role": "system", "content": "你是一个熟知加勒比地区的资深新闻分析师。"},
        {"role": "user", "content": f"通读新闻: {text}，{questions_prompt}"}
    ]
    response_content = call_gpt_api(kinds, 500, 3)
    try:
        answers = json.loads(response_content)
        return (
            answers.get("问题一", "未知"),
            answers.get("问题二", "未知"),
            answers.get("问题三", "未知"),
            answers.get("问题四", "未知"),
            answers.get("问题五", "未知"),
            answers.get("问题六", "未知"),
            answers.get("问题七", "未知")
        )
    except json.JSONDecodeError:
        print("问题回答过程出现问题")
        return ("未知", "未知", "未知", "未知", "未知", "未知", "未知")

# 遍历每一行新闻，进行问题回答
for i, row in df.iterrows():
    print(f"正在处理第{i + 1}/{len(df)}条新闻...")

    # 获取新闻的正文和标题
    text = row['正文'] + ' ' + row['新闻标题']

    # 获取问题一到问题七的答案
    answers = get_answers(text)

    # 将答案写入 DataFrame
    df.at[i, '问题一'] = answers[0]
    df.at[i, '问题二'] = answers[1]
    df.at[i, '问题三'] = answers[2]
    df.at[i, '问题四'] = answers[3]
    df.at[i, '问题五'] = answers[4]
    df.at[i, '问题六'] = answers[5]
    df.at[i, '问题七'] = answers[6]

# 保存修改后的 DataFrame 到新的 Excel 文件
new_file_path = r"C:\Users\86155\Desktop\学习\6.xlsx"
df.to_excel(new_file_path, index=False)
print(f"所有新闻处理完毕，已保存至：{new_file_path}")

正在处理第1/391条新闻...
正在处理第2/391条新闻...
正在处理第3/391条新闻...
正在处理第4/391条新闻...
正在处理第5/391条新闻...
正在处理第6/391条新闻...
正在处理第7/391条新闻...
正在处理第8/391条新闻...
正在处理第9/391条新闻...
正在处理第10/391条新闻...
正在处理第11/391条新闻...
正在处理第12/391条新闻...
正在处理第13/391条新闻...
正在处理第14/391条新闻...
正在处理第15/391条新闻...
正在处理第16/391条新闻...
正在处理第17/391条新闻...
正在处理第18/391条新闻...
问题回答过程出现问题
正在处理第19/391条新闻...
正在处理第20/391条新闻...
正在处理第21/391条新闻...
正在处理第22/391条新闻...
问题回答过程出现问题
正在处理第23/391条新闻...
问题回答过程出现问题
正在处理第24/391条新闻...
正在处理第25/391条新闻...
正在处理第26/391条新闻...
正在处理第27/391条新闻...
正在处理第28/391条新闻...
正在处理第29/391条新闻...
正在处理第30/391条新闻...
问题回答过程出现问题
正在处理第31/391条新闻...
问题回答过程出现问题
正在处理第32/391条新闻...
正在处理第33/391条新闻...
正在处理第34/391条新闻...
正在处理第35/391条新闻...
正在处理第36/391条新闻...
正在处理第37/391条新闻...
正在处理第38/391条新闻...
问题回答过程出现问题
正在处理第39/391条新闻...
正在处理第40/391条新闻...
问题回答过程出现问题
正在处理第41/391条新闻...
问题回答过程出现问题
正在处理第42/391条新闻...
正在处理第43/391条新闻...
正在处理第44/391条新闻...
正在处理第45/391条新闻...
正在处理第46/391条新闻...
正在处理第47/391条新闻...
正在处理第48/391条新闻...
正在处理第49/391条新闻...
正在处理第50/391条新闻...
正在处理第51/391条新闻...
正在处